In [396]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [397]:
zri = pd.read_csv('../data/zri_multifamily_v2.csv')

In [398]:
zri['zip'] = zri['zip'].astype('string')
zri['zip'] = ["0"+z if len(z)<5 else z for z in zri['zip']]
zri['year'] = pd.to_datetime(zri['year-month'],format="%Y-%m").dt.year
zri['year'] = zri['year'].astype('string')
zri['year-month'] = pd.to_datetime(zri['year-month'],format="%Y-%m")

In [399]:
zri[zri['State']=='NJ'].head()

,zip,City,State,Metro,CountyName,year-month,zri,year
13536,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-01-01,1415.0,2014
13537,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-02-01,1415.0,2014
13538,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-03-01,1433.0,2014
13539,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-04-01,1434.0,2014
13540,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-05-01,1436.0,2014


In [400]:
zri.shape

(93744, 8)

In [401]:
#zri['CountyName'][~zri['CountyName'].map(lambda s: s.endswith(' County'))].value_counts()

In [402]:
bds = pd.read_csv('../../../zillow_orientation/bds2018_cty.csv')

In [403]:
bds.head(2)

,year,st,cty,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,job_creation,job_creation_births,job_creation_continuers,job_creation_rate_births,job_creation_rate,job_destruction,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,1,1,396,402,5343,5049,95,24.643,62,16.083,1437,555,882,10.992,28.461,870,742,128,14.696,17.231,567,11.230,34.462,40,40,104
1,1978,1,3,1435,1534,14527,13962,312,21.239,182,12.389,3354,1228,2126,8.795,24.022,2319,1143,1176,8.187,16.609,1035,7.413,33.219,144,145,539


In [404]:
bds.shape

(133250, 27)

In [405]:
bds[['year','st','cty']].duplicated().sum()

0

In [406]:
bds[['year','cty']].duplicated().sum()

119720

In [407]:
bds = bds[['year','st','cty','firms','job_creation_rate','job_destruction_rate']]

In [408]:
bds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133250 entries, 0 to 133249
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   year                  133250 non-null  int64 
 1   st                    133250 non-null  int64 
 2   cty                   133250 non-null  int64 
 3   firms                 133250 non-null  object
 4   job_creation_rate     133250 non-null  object
 5   job_destruction_rate  133250 non-null  object
dtypes: int64(3), object(3)
memory usage: 6.1+ MB


In [409]:
#bds[(bds['year']<=2019) & (bds['year']>=2014)]
bds.columns = ['year','st','cty','total_firms','job_creation_rate','job_destruction_rate']
bds

,year,st,cty,total_firms,job_creation_rate,job_destruction_rate
0,1978,1,1,396,28.461,17.231
1,1978,1,3,1435,24.022,16.609
2,1978,1,5,442,24.467,11.637
3,1978,1,7,187,20.133,19.367
4,1978,1,9,413,20.209,12.750
...,...,...,...,...,...,...
133245,2018,56,45,177,11.053,12.426
133246,2018,56,998,(D),(D),(D)
133247,2018,56,999,134,12.920,12.775
133248,2018,97,998,0,.,.


In [410]:
bds_startup = pd.read_csv('../../../zillow_orientation/bds2018_cty_fage.csv')

In [411]:
#(bds_startup['year']<=2019) & (bds_startup['year']>=2014) 
bds_startup = bds_startup[(bds_startup['fage']=='a) 0')]

In [412]:
bds_startup

,year,st,cty,fage,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,job_creation,job_creation_births,job_creation_continuers,job_creation_rate_births,job_creation_rate,job_destruction,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,1,1,a) 0,81,81,421,211,81,200.000,0,0.000,421,421,0,200.000,200.000,0,0,0,0.000,0.000,421,200.000,0.000,0,0,0
5,1978,1,3,a) 0,259,261,991,496,261,200.000,0,0.000,991,991,0,200.000,200.000,0,0,0,0.000,0.000,991,200.000,0.000,0,0,0
10,1978,1,5,a) 0,51,51,214,107,51,200.000,0,0.000,214,214,0,200.000,200.000,0,0,0,0.000,0.000,214,200.000,0.000,0,0,0
15,1978,1,7,a) 0,27,27,159,80,27,200.000,0,0.000,159,159,0,200.000,200.000,0,0,0,0.000,0.000,159,200.000,0.000,0,0,0
20,1978,1,9,a) 0,56,56,225,113,56,200.000,0,0.000,225,225,0,200.000,200.000,0,0,0,0.000,0.000,225,200.000,0.000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666225,2018,56,45,a) 0,10,10,28,14,10,200.000,0,0.000,28,28,0,200.000,200.000,0,0,0,0.000,0.000,28,200.000,0.000,0,0,0
666230,2018,56,998,a) 0,0,0,0,0,0,NaN,0,NaN,0,0,0,.,.,0,0,0,.,.,0,.,.,0,0,0
666235,2018,56,999,a) 0,0,0,0,0,0,NaN,0,NaN,0,0,0,.,.,0,0,0,.,.,0,.,.,0,0,0
666240,2018,97,998,a) 0,0,0,0,0,0,NaN,0,NaN,0,0,0,.,.,0,0,0,.,.,0,.,.,0,0,0


In [413]:
bds_startup = bds_startup[['year','st','cty','firms']]

In [414]:
bds_startup.columns = ['year','st','cty','startup_firms']

In [415]:
bds_merged = pd.merge(bds,bds_startup,how='inner')

In [416]:
bds_merged = bds_merged[~bds_merged['total_firms'].map(lambda s: s.startswith('('))]

In [417]:
bds_merged.shape

(132527, 7)

In [418]:
bds_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132527 entries, 0 to 133249
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   year                  132527 non-null  int64 
 1   st                    132527 non-null  int64 
 2   cty                   132527 non-null  int64 
 3   total_firms           132527 non-null  object
 4   job_creation_rate     132527 non-null  object
 5   job_destruction_rate  132527 non-null  object
 6   startup_firms         132527 non-null  object
dtypes: int64(3), object(4)
memory usage: 8.1+ MB


In [419]:
bds_merged['year_usable'] = bds_merged['year']+2

In [420]:
bds_merged['year_usable'].unique()

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [421]:
bds_merged = bds_merged[(bds_merged['year_usable']<=2019) & (bds_merged['year_usable']>=2014)]

In [422]:
bds_merged

,year,st,cty,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable
110500,2012,1,1,731,10.885,11.053,35,2014
110501,2012,1,3,3718,14.967,12.992,280,2014
110502,2012,1,5,413,7.714,14.761,9,2014
110503,2012,1,7,250,12.393,9.457,12,2014
110504,2012,1,9,601,13.101,10.389,26,2014
...,...,...,...,...,...,...,...,...
129994,2017,56,43,307,12.816,16.938,11,2019
129995,2017,56,45,187,12.464,14.893,7,2019
129997,2017,56,999,128,21.946,18.451,0,2019
129998,2017,97,998,0,0.000,200.000,0,2019


In [423]:
cty_map = pd.read_csv('../../../zillow_orientation/label_county.csv')

In [424]:
#cty_map = cty_map.drop(['geoid','availability'],axis=1)

In [425]:
cty_map

,state,county,geoid,name,availability
0,1,0,1000,Alabama,2
1,2,0,2000,Alaska,2
2,4,0,4000,Arizona,2
3,5,0,5000,Arkansas,2
4,6,0,6000,California,2
...,...,...,...,...,...
3275,56,39,56039,"Teton, WY",1
3276,56,41,56041,"Uinta, WY",1
3277,56,43,56043,"Washakie, WY",1
3278,56,45,56045,"Weston, WY",1


In [426]:
bds_merged = pd.merge(bds_merged,cty_map,how='left',left_on=['st','cty'],right_on=['state','county'])

In [427]:
bds_merged = bds_merged.dropna()
bds_merged = bds_merged.drop(['state','county','st','cty','availability'],axis=1)

In [428]:
bds_merged['_state'] = bds_merged['name'].map(lambda s: s[-2:])

In [429]:
bds_merged['_county_label'] = bds_merged['name'].map(lambda s: s.split(',')[0])

In [430]:
bds_merged

,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable,geoid,name,_state,_county_label
0,2012,731,10.885,11.053,35,2014,1001.0,"Autauga, AL",AL,Autauga
1,2012,3718,14.967,12.992,280,2014,1003.0,"Baldwin, AL",AL,Baldwin
2,2012,413,7.714,14.761,9,2014,1005.0,"Barbour, AL",AL,Barbour
3,2012,250,12.393,9.457,12,2014,1007.0,"Bibb, AL",AL,Bibb
4,2012,601,13.101,10.389,26,2014,1009.0,"Blount, AL",AL,Blount
...,...,...,...,...,...,...,...,...,...,...
19355,2017,1627,13.632,11.981,143,2019,56039.0,"Teton, WY",WY,Teton
19356,2017,460,8.674,10.371,23,2019,56041.0,"Uinta, WY",WY,Uinta
19357,2017,307,12.816,16.938,11,2019,56043.0,"Washakie, WY",WY,Washakie
19358,2017,187,12.464,14.893,7,2019,56045.0,"Weston, WY",WY,Weston


In [431]:
#bds_merged.to_csv('./../data/bds_engineered_features_yearly_county.csv')

In [432]:
#bds_merged[bds_merged['name'].map(lambda s: s.endswith('NJ'))]

In [433]:
#bds_merged['name'].value_counts()

In [434]:
#bds_merged[bds_merged['name'].map(lambda s: s.endswith('VA'))].head(500)

In [435]:
#bds_merged[bds_merged['name'].map(lambda s: s.endswith('AK'))].head(500)

In [436]:
b = bds_merged[['_state','_county_label']]
b.columns = ['bds_state','bds_county']
z = zri[['State','CountyName']]
z.columns = ['zillow_state','zillow_county']

In [437]:
z_b_state = pd.merge(z,b,how='left',left_on='zillow_state',right_on='bds_state')
z_b_state

,zillow_state,zillow_county,bds_state,bds_county
0,MA,Hampden County,MA,Barnstable
1,MA,Hampden County,MA,Berkshire
2,MA,Hampden County,MA,Bristol
3,MA,Hampden County,MA,Dukes
4,MA,Hampden County,MA,Essex
...,...,...,...,...
42468043,AK,Matanuska Susitna Borough,AK,Valdez-Cordova
42468044,AK,Matanuska Susitna Borough,AK,Wrangell
42468045,AK,Matanuska Susitna Borough,AK,Wrangell-Petersburg
42468046,AK,Matanuska Susitna Borough,AK,Yakutat


In [438]:
z_b_state['zillow_county_2'] = z_b_state['zillow_county'].map(lambda s: s.split(" County")[0])

In [439]:
z_b_state['zillow_county_2'] = z_b_state['zillow_county_2'].map(lambda s: s.split(" Borough")[0])

In [440]:
z_b_state.loc[z_b_state['bds_county']=='St. Louis','bds_county'] = 'Saint Louis'

In [441]:
z_b_state.loc[z_b_state['bds_county']=='St. Louis city','bds_county'] = 'Saint Louis City'

In [442]:
z_b_state.loc[z_b_state['bds_county']=='Matanuska-Susitna','bds_county'] = 'Matanuska Susitna'

In [443]:
z_b_state.loc[z_b_state['bds_county']=='Prince George\'s','bds_county'] = 'Prince Georges'

In [444]:
z_b_state['bds_county'] = z_b_state['bds_county'].map(lambda s: s.lower())

In [445]:
z_b_state['zillow_county_2'] = z_b_state['zillow_county_2'].map(lambda s: s.lower())

In [446]:
z = z_b_state[['zillow_state','zillow_county','zillow_county_2']]
z = z.drop_duplicates()

In [447]:
b = z_b_state[['zillow_state','bds_county']]
b = b.drop_duplicates()

In [448]:
county_map = pd.merge(z,b,how='left',left_on=['zillow_state','zillow_county_2'],right_on=['zillow_state','bds_county'])

In [449]:
county_map.shape

(237, 4)

In [450]:
z.shape

(237, 3)

In [451]:
b.shape

(2690, 2)

In [452]:
bds_merged

,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable,geoid,name,_state,_county_label
0,2012,731,10.885,11.053,35,2014,1001.0,"Autauga, AL",AL,Autauga
1,2012,3718,14.967,12.992,280,2014,1003.0,"Baldwin, AL",AL,Baldwin
2,2012,413,7.714,14.761,9,2014,1005.0,"Barbour, AL",AL,Barbour
3,2012,250,12.393,9.457,12,2014,1007.0,"Bibb, AL",AL,Bibb
4,2012,601,13.101,10.389,26,2014,1009.0,"Blount, AL",AL,Blount
...,...,...,...,...,...,...,...,...,...,...
19355,2017,1627,13.632,11.981,143,2019,56039.0,"Teton, WY",WY,Teton
19356,2017,460,8.674,10.371,23,2019,56041.0,"Uinta, WY",WY,Uinta
19357,2017,307,12.816,16.938,11,2019,56043.0,"Washakie, WY",WY,Washakie
19358,2017,187,12.464,14.893,7,2019,56045.0,"Weston, WY",WY,Weston


In [453]:
bds_merged.loc[bds_merged['_county_label']=='St. Louis','_county_label'] = 'Saint Louis'

In [454]:
bds_merged.loc[bds_merged['_county_label']=='St. Louis city','_county_label'] = 'Saint Louis City'

In [455]:
bds_merged.loc[bds_merged['_county_label']=='Matanuska-Susitna','_county_label'] = 'Matanuska Susitna'

In [456]:
bds_merged.loc[bds_merged['_county_label']=='Prince George\'s','_county_label'] = 'Prince Georges'

In [457]:
bds_merged['_county_label'] = bds_merged['_county_label'].map(lambda s: s.lower())

In [458]:
bds_final = pd.merge(county_map,bds_merged,how='left',left_on=['zillow_state','bds_county'],right_on=['_state','_county_label'])

In [459]:
county_map.head(2)

,zillow_state,zillow_county,zillow_county_2,bds_county
0,MA,Hampden County,hampden,hampden
1,MA,Worcester County,worcester,worcester


In [460]:
bds_merged.head(2)

,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable,geoid,name,_state,_county_label
0,2012,731,10.885,11.053,35,2014,1001.0,"Autauga, AL",AL,autauga
1,2012,3718,14.967,12.992,280,2014,1003.0,"Baldwin, AL",AL,baldwin


In [461]:
bds_final.isna().sum()

zillow_state            0
zillow_county           0
zillow_county_2         0
bds_county              0
year                    0
total_firms             0
job_creation_rate       0
job_destruction_rate    0
startup_firms           0
year_usable             0
geoid                   0
name                    0
_state                  0
_county_label           0
dtype: int64

In [462]:
bds_final

,zillow_state,zillow_county,zillow_county_2,bds_county,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable,geoid,name,_state,_county_label
0,MA,Hampden County,hampden,hampden,2012,7376,9.972,9.108,393,2014,25013.0,"Hampden, MA",MA,hampden
1,MA,Hampden County,hampden,hampden,2013,7340,10.221,8.228,383,2015,25013.0,"Hampden, MA",MA,hampden
2,MA,Hampden County,hampden,hampden,2014,7287,8.516,9.429,371,2016,25013.0,"Hampden, MA",MA,hampden
3,MA,Hampden County,hampden,hampden,2015,7170,10.373,8.573,332,2017,25013.0,"Hampden, MA",MA,hampden
4,MA,Hampden County,hampden,hampden,2016,7268,11.373,9.069,436,2018,25013.0,"Hampden, MA",MA,hampden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,AK,Matanuska Susitna Borough,matanuska susitna,matanuska susitna,2013,1581,13.809,12.478,124,2015,2170.0,"Matanuska-Susitna, AK",AK,matanuska susitna
1418,AK,Matanuska Susitna Borough,matanuska susitna,matanuska susitna,2014,1618,14.033,11.313,146,2016,2170.0,"Matanuska-Susitna, AK",AK,matanuska susitna
1419,AK,Matanuska Susitna Borough,matanuska susitna,matanuska susitna,2015,1658,14.622,10.717,140,2017,2170.0,"Matanuska-Susitna, AK",AK,matanuska susitna
1420,AK,Matanuska Susitna Borough,matanuska susitna,matanuska susitna,2016,1720,13.782,10.990,148,2018,2170.0,"Matanuska-Susitna, AK",AK,matanuska susitna


In [463]:
bds_final = bds_final.drop(['zillow_county_2','bds_county','name','_state','_county_label','geoid','year'],axis=1)

In [464]:
bds_final

,zillow_state,zillow_county,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable
0,MA,Hampden County,7376,9.972,9.108,393,2014
1,MA,Hampden County,7340,10.221,8.228,383,2015
2,MA,Hampden County,7287,8.516,9.429,371,2016
3,MA,Hampden County,7170,10.373,8.573,332,2017
4,MA,Hampden County,7268,11.373,9.069,436,2018
...,...,...,...,...,...,...,...
1417,AK,Matanuska Susitna Borough,1581,13.809,12.478,124,2015
1418,AK,Matanuska Susitna Borough,1618,14.033,11.313,146,2016
1419,AK,Matanuska Susitna Borough,1658,14.622,10.717,140,2017
1420,AK,Matanuska Susitna Borough,1720,13.782,10.990,148,2018


In [465]:
bds_final['zillow_county'].nunique()

214

In [466]:
zri['CountyName'].nunique()

214

In [467]:
county_map

,zillow_state,zillow_county,zillow_county_2,bds_county
0,MA,Hampden County,hampden,hampden
1,MA,Worcester County,worcester,worcester
2,MA,Middlesex County,middlesex,middlesex
3,MA,Essex County,essex,essex
4,MA,Norfolk County,norfolk,norfolk
5,MA,Suffolk County,suffolk,suffolk
6,MA,Plymouth County,plymouth,plymouth
7,MA,Bristol County,bristol,bristol
8,RI,Newport County,newport,newport
9,RI,Providence County,providence,providence


In [469]:
bds_final

,zillow_state,zillow_county,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable
0,MA,Hampden County,7376,9.972,9.108,393,2014
1,MA,Hampden County,7340,10.221,8.228,383,2015
2,MA,Hampden County,7287,8.516,9.429,371,2016
3,MA,Hampden County,7170,10.373,8.573,332,2017
4,MA,Hampden County,7268,11.373,9.069,436,2018
...,...,...,...,...,...,...,...
1417,AK,Matanuska Susitna Borough,1581,13.809,12.478,124,2015
1418,AK,Matanuska Susitna Borough,1618,14.033,11.313,146,2016
1419,AK,Matanuska Susitna Borough,1658,14.622,10.717,140,2017
1420,AK,Matanuska Susitna Borough,1720,13.782,10.990,148,2018


In [480]:
bds_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422 entries, 0 to 1421
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   zillow_state          1422 non-null   object
 1   zillow_county         1422 non-null   object
 2   total_firms           1422 non-null   object
 3   job_creation_rate     1422 non-null   object
 4   job_destruction_rate  1422 non-null   object
 5   startup_firms         1422 non-null   object
 6   year_usable           1422 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 88.9+ KB


In [482]:
bds_final['year_usable'] = bds_final['year_usable'].astype('string')

In [481]:
zri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93744 entries, 0 to 93743
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   zip         93744 non-null  object        
 1   City        93744 non-null  object        
 2   State       93744 non-null  object        
 3   Metro       93672 non-null  object        
 4   CountyName  93744 non-null  object        
 5   year-month  93744 non-null  datetime64[ns]
 6   zri         93744 non-null  float64       
 7   year        93744 non-null  string        
dtypes: datetime64[ns](1), float64(1), object(5), string(1)
memory usage: 5.7+ MB


In [483]:
zillow_bds = pd.merge(zri,bds_final,how='left',left_on=['CountyName','State','year'],right_on=['zillow_county','zillow_state','year_usable'])

In [484]:
zillow_bds

,zip,City,State,Metro,CountyName,year-month,zri,year,zillow_state,zillow_county,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable
0,01013,Chicopee,MA,Springfield,Hampden County,2014-01-01,928.0,2014,MA,Hampden County,7376,9.972,9.108,393,2014
1,01013,Chicopee,MA,Springfield,Hampden County,2014-02-01,931.0,2014,MA,Hampden County,7376,9.972,9.108,393,2014
2,01013,Chicopee,MA,Springfield,Hampden County,2014-03-01,934.0,2014,MA,Hampden County,7376,9.972,9.108,393,2014
3,01013,Chicopee,MA,Springfield,Hampden County,2014-04-01,929.0,2014,MA,Hampden County,7376,9.972,9.108,393,2014
4,01013,Chicopee,MA,Springfield,Hampden County,2014-05-01,929.0,2014,MA,Hampden County,7376,9.972,9.108,393,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93739,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-08-01,1277.0,2019,AK,Matanuska Susitna Borough,1750,16.772,16.359,161,2019
93740,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-09-01,1271.0,2019,AK,Matanuska Susitna Borough,1750,16.772,16.359,161,2019
93741,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-10-01,1299.0,2019,AK,Matanuska Susitna Borough,1750,16.772,16.359,161,2019
93742,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-11-01,1261.5,2019,AK,Matanuska Susitna Borough,1750,16.772,16.359,161,2019


In [485]:
zillow_bds = zillow_bds.drop(['zillow_state','zillow_county','year_usable'],axis=1)

In [486]:
zillow_bds.isna().sum()

zip                      0
City                     0
State                    0
Metro                   72
CountyName               0
year-month               0
zri                      0
year                     0
total_firms              0
job_creation_rate        0
job_destruction_rate     0
startup_firms            0
dtype: int64

In [487]:
zillow_bds

,zip,City,State,Metro,CountyName,year-month,zri,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms
0,01013,Chicopee,MA,Springfield,Hampden County,2014-01-01,928.0,2014,7376,9.972,9.108,393
1,01013,Chicopee,MA,Springfield,Hampden County,2014-02-01,931.0,2014,7376,9.972,9.108,393
2,01013,Chicopee,MA,Springfield,Hampden County,2014-03-01,934.0,2014,7376,9.972,9.108,393
3,01013,Chicopee,MA,Springfield,Hampden County,2014-04-01,929.0,2014,7376,9.972,9.108,393
4,01013,Chicopee,MA,Springfield,Hampden County,2014-05-01,929.0,2014,7376,9.972,9.108,393
...,...,...,...,...,...,...,...,...,...,...,...,...
93739,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-08-01,1277.0,2019,1750,16.772,16.359,161
93740,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-09-01,1271.0,2019,1750,16.772,16.359,161
93741,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-10-01,1299.0,2019,1750,16.772,16.359,161
93742,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-11-01,1261.5,2019,1750,16.772,16.359,161


In [488]:
zillow_bds.to_csv('./../data/bds_engineered_features.csv')

In [ ]:
# for state in z_b_state['zillow_state'].unique():
#     temp = z_b_state[z_b_state['zillow_state']==state]
#     print(state)
#     print(sorted(set(temp['zillow_county_2'])))
#     print("\n")
#     print(sorted(set(temp['bds_county'])))
#     print("\n")

In [101]:
# for state in z_b_state['zillow_state'].unique():
#     temp = z_b_state[z_b_state['zillow_state']==state]
#     print(state)
#     print(sorted(set(temp['zillow_county_2'])))
#     print("\n")
#     print(sorted(set(temp['bds_county'])))
#     print("\n")

MA
['Bristol', 'Essex', 'Hampden', 'Middlesex', 'Norfolk', 'Plymouth', 'Suffolk', 'Worcester']


['Barnstable', 'Berkshire', 'Bristol', 'Dukes', 'Essex', 'Franklin', 'Hampden', 'Hampshire', 'Middlesex', 'Nantucket', 'Norfolk', 'Plymouth', 'Suffolk', 'Worcester']


RI
['Kent', 'Newport', 'Providence', 'Washington']


['Bristol', 'Kent', 'Newport', 'Providence', 'Washington']


NH
['Hillsborough', 'Merrimack', 'Rockingham', 'Strafford']


['Belknap', 'Carroll', 'Cheshire', 'Coos', 'Grafton', 'Hillsborough', 'Merrimack', 'Rockingham', 'Strafford', 'Sullivan']


ME
['Androscoggin', 'Cumberland', 'Kennebec', 'Penobscot', 'Somerset']


['Androscoggin', 'Aroostook', 'Cumberland', 'Franklin', 'Hancock', 'Kennebec', 'Knox', 'Lincoln', 'Oxford', 'Penobscot', 'Piscataquis', 'Sagadahoc', 'Somerset', 'Waldo', 'Washington', 'York']


CT
['Fairfield', 'Hartford', 'Litchfield', 'Middlesex', 'New Haven', 'New London']


['Fairfield', 'Hartford', 'Litchfield', 'Middlesex', 'New Haven', 'New London', 'To

FL
['Brevard', 'Broward', 'Duval', 'Hillsborough', 'Lee', 'Manatee', 'Miami-Dade', 'Orange', 'Palm Beach', 'Pinellas', 'Sarasota', 'Seminole']


['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia', 'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'Indian River', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin', 'Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Palm Beach', 'Pasco', 'Pinellas', 'Polk', 'Putnam', 'Santa Rosa', 'Sarasota', 'Seminole', 'St. Johns', 'St. Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Volusia', 'Wakulla', 'Walton', 'Washington']


TN
['Davidson', 'Montgomery', 'Shelby', 'Sumner']


['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley'

MN
['Dakota', 'Hennepin', 'Ramsey']


['Aitkin', 'Anoka', 'Becker', 'Beltrami', 'Benton', 'Big Stone', 'Blue Earth', 'Brown', 'Carlton', 'Carver', 'Cass', 'Chippewa', 'Chisago', 'Clay', 'Clearwater', 'Cook', 'Cottonwood', 'Crow Wing', 'Dakota', 'Dodge', 'Douglas', 'Faribault', 'Fillmore', 'Freeborn', 'Goodhue', 'Grant', 'Hennepin', 'Houston', 'Hubbard', 'Isanti', 'Itasca', 'Jackson', 'Kanabec', 'Kandiyohi', 'Kittson', 'Koochiching', 'Lac qui Parle', 'Lake', 'Lake of the Woods', 'Le Sueur', 'Lincoln', 'Lyon', 'Mahnomen', 'Marshall', 'Martin', 'McLeod', 'Meeker', 'Mille Lacs', 'Morrison', 'Mower', 'Murray', 'Nicollet', 'Nobles', 'Norman', 'Olmsted', 'Otter Tail', 'Pennington', 'Pine', 'Pipestone', 'Polk', 'Pope', 'Ramsey', 'Red Lake', 'Redwood', 'Renville', 'Rice', 'Rock', 'Roseau', 'Scott', 'Sherburne', 'Sibley', 'St. Louis', 'Stearns', 'Steele', 'Stevens', 'Swift', 'Todd', 'Traverse', 'Wabasha', 'Wadena', 'Waseca', 'Washington', 'Watonwan', 'Wilkin', 'Winona', 'Wright', 'Yellow Medicin

['Anderson', 'Andrews', 'Angelina', 'Aransas', 'Archer', 'Armstrong', 'Atascosa', 'Austin', 'Bailey', 'Bandera', 'Bastrop', 'Baylor', 'Bee', 'Bell', 'Bexar', 'Blanco', 'Borden', 'Bosque', 'Bowie', 'Brazoria', 'Brazos', 'Brewster', 'Briscoe', 'Brooks', 'Brown', 'Burleson', 'Burnet', 'Caldwell', 'Calhoun', 'Callahan', 'Cameron', 'Camp', 'Carson', 'Cass', 'Castro', 'Chambers', 'Cherokee', 'Childress', 'Clay', 'Cochran', 'Coke', 'Coleman', 'Collin', 'Collingsworth', 'Colorado', 'Comal', 'Comanche', 'Concho', 'Cooke', 'Coryell', 'Cottle', 'Crane', 'Crockett', 'Crosby', 'Culberson', 'Dallam', 'Dallas', 'Dawson', 'DeWitt', 'Deaf Smith', 'Delta', 'Denton', 'Dickens', 'Dimmit', 'Donley', 'Duval', 'Eastland', 'Ector', 'Edwards', 'El Paso', 'Ellis', 'Erath', 'Falls', 'Fannin', 'Fayette', 'Fisher', 'Floyd', 'Foard', 'Fort Bend', 'Franklin', 'Freestone', 'Frio', 'Gaines', 'Galveston', 'Garza', 'Gillespie', 'Glasscock', 'Goliad', 'Gonzales', 'Gray', 'Grayson', 'Gregg', 'Grimes', 'Guadalupe', 'Hale',

In [142]:
# MA
# ['Bristol', 'Essex', 'Hampden', 'Middlesex', 'Norfolk', 'Plymouth', 'Suffolk', 'Worcester']


# ['Barnstable', 'Berkshire', 'Bristol', 'Dukes', 'Essex', 'Franklin', 'Hampden', 'Hampshire', 'Middlesex', 'Nantucket', 'Norfolk', 'Plymouth', 'Suffolk', 'Worcester']


# RI
# ['Kent', 'Newport', 'Providence', 'Washington']


# ['Bristol', 'Kent', 'Newport', 'Providence', 'Washington']


# NH
# ['Hillsborough', 'Merrimack', 'Rockingham', 'Strafford']


# ['Belknap', 'Carroll', 'Cheshire', 'Coos', 'Grafton', 'Hillsborough', 'Merrimack', 'Rockingham', 'Strafford', 'Sullivan']


# ME
# ['Androscoggin', 'Cumberland', 'Kennebec', 'Penobscot', 'Somerset']


# ['Androscoggin', 'Aroostook', 'Cumberland', 'Franklin', 'Hancock', 'Kennebec', 'Knox', 'Lincoln', 'Oxford', 'Penobscot', 'Piscataquis', 'Sagadahoc', 'Somerset', 'Waldo', 'Washington', 'York']


# CT
# ['Fairfield', 'Hartford', 'Litchfield', 'Middlesex', 'New Haven', 'New London']


# ['Fairfield', 'Hartford', 'Litchfield', 'Middlesex', 'New Haven', 'New London', 'Tolland', 'Windham']


# NJ
# ['Essex', 'Hudson', 'Middlesex', 'Morris', 'Passaic', 'Union']


# ['Atlantic', 'Bergen', 'Burlington', 'Camden', 'Cape May', 'Cumberland', 'Essex', 'Gloucester', 'Hudson', 'Hunterdon', 'Mercer', 'Middlesex', 'Monmouth', 'Morris', 'Ocean', 'Passaic', 'Salem', 'Somerset', 'Sussex', 'Union', 'Warren']


# NY
# ['Albany', 'Bronx', 'Broome', 'Chautauqua', 'Dutchess', 'Erie', 'Genesee', 'Jefferson', 'Kings', 'Monroe', 'Nassau', 'New York', 'Niagara', 'Oneida', 'Onondaga', 'Orange', 'Queens', 'Rensselaer', 'Richmond', 'Rockland', 'Saratoga', 'Schenectady', 'Tompkins', 'Ulster', 'Westchester']


# ['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga', 'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia', 'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin', 'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson', 'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery', 'Nassau', 'New York', 'Niagara', 'Oneida', 'Onondaga', 'Ontario', 'Orange', 'Orleans', 'Oswego', 'Otsego', 'Putnam', 'Queens', 'Rensselaer', 'Richmond', 'Rockland', 'Saratoga', 'Schenectady', 'Schoharie', 'Schuyler', 'Seneca', 'St. Lawrence', 'Steuben', 'Suffolk', 'Sullivan', 'Tioga', 'Tompkins', 'Ulster', 'Warren', 'Washington', 'Wayne', 'Westchester', 'Wyoming', 'Yates']


# PA
# ['Allegheny', 'Bucks', 'Carbon', 'Cumberland', 'Erie', 'Lancaster', 'Lehigh', 'Lycoming', 'Montgomery', 'Northampton', 'Philadelphia', 'Warren', 'York']


# ['Adams', 'Allegheny', 'Armstrong', 'Beaver', 'Bedford', 'Berks', 'Blair', 'Bradford', 'Bucks', 'Butler', 'Cambria', 'Cameron', 'Carbon', 'Centre', 'Chester', 'Clarion', 'Clearfield', 'Clinton', 'Columbia', 'Crawford', 'Cumberland', 'Dauphin', 'Delaware', 'Elk', 'Erie', 'Fayette', 'Forest', 'Franklin', 'Fulton', 'Greene', 'Huntingdon', 'Indiana', 'Jefferson', 'Juniata', 'Lackawanna', 'Lancaster', 'Lawrence', 'Lebanon', 'Lehigh', 'Luzerne', 'Lycoming', 'McKean', 'Mercer', 'Mifflin', 'Monroe', 'Montgomery', 'Montour', 'Northampton', 'Northumberland', 'Perry', 'Philadelphia', 'Pike', 'Potter', 'Schuylkill', 'Snyder', 'Somerset', 'Sullivan', 'Susquehanna', 'Tioga', 'Union', 'Venango', 'Warren', 'Washington', 'Wayne', 'Westmoreland', 'Wyoming', 'York']


# DC
# ['District of Columbia']


# ['District of Columbia']


# VA
# ['Albemarle', 'Alexandria City', 'Arlington', 'Chesterfield', 'Fairfax', 'Hampton City', 'Henrico', 'Loudoun', 'Newport News City', 'Norfolk City', 'Petersburg City', 'Prince William', 'Richmond City', 'Virginia Beach City']


# ['Accomack', 'Albemarle', 'Alexandria city', 'Alleghany', 'Amelia', 'Amherst', 'Appomattox', 'Arlington', 'Augusta', 'Bath', 'Bedford', 'Bland', 'Botetourt', 'Bristol city', 'Brunswick', 'Buchanan', 'Buckingham', 'Buena Vista city', 'Campbell', 'Caroline', 'Carroll', 'Charles City', 'Charlotte', 'Charlottesville city', 'Chesapeake city', 'Chesterfield', 'Clarke', 'Colonial Heights city', 'Covington city', 'Craig', 'Culpeper', 'Cumberland', 'Danville city', 'Dickenson', 'Dinwiddie', 'Emporia city', 'Essex', 'Fairfax', 'Fairfax city', 'Falls Church city', 'Fauquier', 'Floyd', 'Fluvanna', 'Franklin', 'Franklin city', 'Frederick', 'Fredericksburg city', 'Galax city', 'Giles', 'Gloucester', 'Goochland', 'Grayson', 'Greene', 'Greensville', 'Halifax', 'Hampton city', 'Hanover', 'Harrisonburg city', 'Henrico', 'Henry', 'Highland', 'Hopewell city', 'Isle of Wight', 'James City', 'King George', 'King William', 'King and Queen', 'Lancaster', 'Lee', 'Lexington city', 'Loudoun', 'Louisa', 'Lunenburg', 'Lynchburg city', 'Madison', 'Manassas Park city', 'Manassas city', 'Martinsville city', 'Mathews', 'Mecklenburg', 'Middlesex', 'Montgomery', 'Nelson', 'New Kent', 'Newport News city', 'Norfolk city', 'Northampton', 'Northumberland', 'Norton city', 'Nottoway', 'Orange', 'Page', 'Patrick', 'Petersburg city', 'Pittsylvania', 'Poquoson city', 'Portsmouth city', 'Powhatan', 'Prince Edward', 'Prince George', 'Prince William', 'Pulaski', 'Radford city', 'Rappahannock', 'Richmond', 'Richmond city', 'Roanoke', 'Roanoke city', 'Rockbridge', 'Rockingham', 'Russell', 'Salem city', 'Scott', 'Shenandoah', 'Smyth', 'Southampton', 'Spotsylvania', 'Stafford', 'Staunton city', 'Suffolk city', 'Surry', 'Sussex', 'Tazewell', 'Virginia Beach city', 'Warren', 'Washington', 'Waynesboro city', 'Westmoreland', 'Williamsburg city', 'Winchester city', 'Wise', 'Wythe', 'York']


# MD
# ['Anne Arundel', 'Baltimore', 'Baltimore City', 'Howard', 'Montgomery', 'Prince Georges']


# ['Allegany', 'Anne Arundel', 'Baltimore', 'Baltimore city', 'Calvert', 'Caroline', 'Carroll', 'Cecil', 'Charles', 'Dorchester', 'Frederick', 'Garrett', 'Harford', 'Howard', 'Kent', 'Montgomery', "Prince George's", "Queen Anne's", 'Somerset', "St. Mary's", 'Talbot', 'Washington', 'Wicomico', 'Worcester']


# WV
# ['Monongalia', 'Ohio']


# ['Barbour', 'Berkeley', 'Boone', 'Braxton', 'Brooke', 'Cabell', 'Calhoun', 'Clay', 'Doddridge', 'Fayette', 'Gilmer', 'Grant', 'Greenbrier', 'Hampshire', 'Hancock', 'Hardy', 'Harrison', 'Jackson', 'Jefferson', 'Kanawha', 'Lewis', 'Lincoln', 'Logan', 'Marion', 'Marshall', 'Mason', 'McDowell', 'Mercer', 'Mineral', 'Mingo', 'Monongalia', 'Monroe', 'Morgan', 'Nicholas', 'Ohio', 'Pendleton', 'Pleasants', 'Pocahontas', 'Preston', 'Putnam', 'Raleigh', 'Randolph', 'Ritchie', 'Roane', 'Summers', 'Taylor', 'Tucker', 'Tyler', 'Upshur', 'Wayne', 'Webster', 'Wetzel', 'Wirt', 'Wood', 'Wyoming']


# NC
# ['Durham', 'Mecklenburg', 'Wake']


# ['Alamance', 'Alexander', 'Alleghany', 'Anson', 'Ashe', 'Avery', 'Beaufort', 'Bertie', 'Bladen', 'Brunswick', 'Buncombe', 'Burke', 'Cabarrus', 'Caldwell', 'Camden', 'Carteret', 'Caswell', 'Catawba', 'Chatham', 'Cherokee', 'Chowan', 'Clay', 'Cleveland', 'Columbus', 'Craven', 'Cumberland', 'Currituck', 'Dare', 'Davidson', 'Davie', 'Duplin', 'Durham', 'Edgecombe', 'Forsyth', 'Franklin', 'Gaston', 'Gates', 'Graham', 'Granville', 'Greene', 'Guilford', 'Halifax', 'Harnett', 'Haywood', 'Henderson', 'Hertford', 'Hoke', 'Hyde', 'Iredell', 'Jackson', 'Johnston', 'Jones', 'Lee', 'Lenoir', 'Lincoln', 'Macon', 'Madison', 'Martin', 'McDowell', 'Mecklenburg', 'Mitchell', 'Montgomery', 'Moore', 'Nash', 'New Hanover', 'Northampton', 'Onslow', 'Orange', 'Pamlico', 'Pasquotank', 'Pender', 'Perquimans', 'Person', 'Pitt', 'Polk', 'Randolph', 'Richmond', 'Robeson', 'Rockingham', 'Rowan', 'Rutherford', 'Sampson', 'Scotland', 'Stanly', 'Stokes', 'Surry', 'Swain', 'Transylvania', 'Tyrrell', 'Union', 'Vance', 'Wake', 'Warren', 'Washington', 'Watauga', 'Wayne', 'Wilkes', 'Wilson', 'Yadkin', 'Yancey']


# SC
# ['Charleston', 'Greenville', 'Richland']


# ['Abbeville', 'Aiken', 'Allendale', 'Anderson', 'Bamberg', 'Barnwell', 'Beaufort', 'Berkeley', 'Calhoun', 'Charleston', 'Cherokee', 'Chester', 'Chesterfield', 'Clarendon', 'Colleton', 'Darlington', 'Dillon', 'Dorchester', 'Edgefield', 'Fairfield', 'Florence', 'Georgetown', 'Greenville', 'Greenwood', 'Hampton', 'Horry', 'Jasper', 'Kershaw', 'Lancaster', 'Laurens', 'Lee', 'Lexington', 'Marion', 'Marlboro', 'McCormick', 'Newberry', 'Oconee', 'Orangeburg', 'Pickens', 'Richland', 'Saluda', 'Spartanburg', 'Sumter', 'Union', 'Williamsburg', 'York']


# GA
# ['Chatham', 'Clarke', 'Cobb', 'Dekalb', 'Fulton', 'Gwinnett']


# ['Appling', 'Atkinson', 'Bacon', 'Baker', 'Baldwin', 'Banks', 'Barrow', 'Bartow', 'Ben Hill', 'Berrien', 'Bibb', 'Bleckley', 'Brantley', 'Brooks', 'Bryan', 'Bulloch', 'Burke', 'Butts', 'Calhoun', 'Camden', 'Candler', 'Carroll', 'Catoosa', 'Charlton', 'Chatham', 'Chattahoochee', 'Chattooga', 'Cherokee', 'Clarke', 'Clay', 'Clayton', 'Clinch', 'Cobb', 'Coffee', 'Colquitt', 'Columbia', 'Cook', 'Coweta', 'Crawford', 'Crisp', 'Dade', 'Dawson', 'DeKalb', 'Decatur', 'Dodge', 'Dooly', 'Dougherty', 'Douglas', 'Early', 'Echols', 'Effingham', 'Elbert', 'Emanuel', 'Evans', 'Fannin', 'Fayette', 'Floyd', 'Forsyth', 'Franklin', 'Fulton', 'Gilmer', 'Glascock', 'Glynn', 'Gordon', 'Grady', 'Greene', 'Gwinnett', 'Habersham', 'Hall', 'Hancock', 'Haralson', 'Harris', 'Hart', 'Heard', 'Henry', 'Houston', 'Irwin', 'Jackson', 'Jasper', 'Jeff Davis', 'Jefferson', 'Jenkins', 'Johnson', 'Jones', 'Lamar', 'Lanier', 'Laurens', 'Lee', 'Liberty', 'Lincoln', 'Long', 'Lowndes', 'Lumpkin', 'Macon', 'Madison', 'Marion', 'McDuffie', 'McIntosh', 'Meriwether', 'Miller', 'Mitchell', 'Monroe', 'Montgomery', 'Morgan', 'Murray', 'Muscogee', 'Newton', 'Oconee', 'Oglethorpe', 'Paulding', 'Peach', 'Pickens', 'Pierce', 'Pike', 'Polk', 'Pulaski', 'Putnam', 'Quitman', 'Rabun', 'Randolph', 'Richmond', 'Rockdale', 'Schley', 'Screven', 'Seminole', 'Spalding', 'Stephens', 'Stewart', 'Sumter', 'Talbot', 'Taliaferro', 'Tattnall', 'Taylor', 'Telfair', 'Terrell', 'Thomas', 'Tift', 'Toombs', 'Towns', 'Treutlen', 'Troup', 'Turner', 'Twiggs', 'Union', 'Upson', 'Walker', 'Walton', 'Ware', 'Warren', 'Washington', 'Wayne', 'Webster', 'Wheeler', 'White', 'Whitfield', 'Wilcox', 'Wilkes', 'Wilkinson', 'Worth']


# FL
# ['Brevard', 'Broward', 'Duval', 'Hillsborough', 'Lee', 'Manatee', 'Miami-Dade', 'Orange', 'Palm Beach', 'Pinellas', 'Sarasota', 'Seminole']


# ['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia', 'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'Indian River', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin', 'Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Palm Beach', 'Pasco', 'Pinellas', 'Polk', 'Putnam', 'Santa Rosa', 'Sarasota', 'Seminole', 'St. Johns', 'St. Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Volusia', 'Wakulla', 'Walton', 'Washington']


# TN
# ['Davidson', 'Montgomery', 'Shelby', 'Sumner']


# ['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley', 'Campbell', 'Cannon', 'Carroll', 'Carter', 'Cheatham', 'Chester', 'Claiborne', 'Clay', 'Cocke', 'Coffee', 'Crockett', 'Cumberland', 'Davidson', 'DeKalb', 'Decatur', 'Dickson', 'Dyer', 'Fayette', 'Fentress', 'Franklin', 'Gibson', 'Giles', 'Grainger', 'Greene', 'Grundy', 'Hamblen', 'Hamilton', 'Hancock', 'Hardeman', 'Hardin', 'Hawkins', 'Haywood', 'Henderson', 'Henry', 'Hickman', 'Houston', 'Humphreys', 'Jackson', 'Jefferson', 'Johnson', 'Knox', 'Lake', 'Lauderdale', 'Lawrence', 'Lewis', 'Lincoln', 'Loudon', 'Macon', 'Madison', 'Marion', 'Marshall', 'Maury', 'McMinn', 'McNairy', 'Meigs', 'Monroe', 'Montgomery', 'Moore', 'Morgan', 'Obion', 'Overton', 'Perry', 'Pickett', 'Polk', 'Putnam', 'Rhea', 'Roane', 'Robertson', 'Rutherford', 'Scott', 'Sequatchie', 'Sevier', 'Shelby', 'Smith', 'Stewart', 'Sullivan', 'Sumner', 'Tipton', 'Trousdale', 'Unicoi', 'Union', 'Van Buren', 'Warren', 'Washington', 'Wayne', 'Weakley', 'White', 'Williamson', 'Wilson']


# KY
# ['Clark', 'Fayette']


# ['Adair', 'Allen', 'Anderson', 'Ballard', 'Barren', 'Bath', 'Bell', 'Boone', 'Bourbon', 'Boyd', 'Boyle', 'Bracken', 'Breathitt', 'Breckinridge', 'Bullitt', 'Butler', 'Caldwell', 'Calloway', 'Campbell', 'Carlisle', 'Carroll', 'Carter', 'Casey', 'Christian', 'Clark', 'Clay', 'Clinton', 'Crittenden', 'Cumberland', 'Daviess', 'Edmonson', 'Elliott', 'Estill', 'Fayette', 'Fleming', 'Floyd', 'Franklin', 'Fulton', 'Gallatin', 'Garrard', 'Grant', 'Graves', 'Grayson', 'Green', 'Greenup', 'Hancock', 'Hardin', 'Harlan', 'Harrison', 'Hart', 'Henderson', 'Henry', 'Hickman', 'Hopkins', 'Jackson', 'Jefferson', 'Jessamine', 'Johnson', 'Kenton', 'Knott', 'Knox', 'Larue', 'Laurel', 'Lawrence', 'Lee', 'Leslie', 'Letcher', 'Lewis', 'Lincoln', 'Livingston', 'Logan', 'Lyon', 'Madison', 'Magoffin', 'Marion', 'Marshall', 'Martin', 'Mason', 'McCracken', 'McCreary', 'McLean', 'Meade', 'Menifee', 'Mercer', 'Metcalfe', 'Monroe', 'Montgomery', 'Morgan', 'Muhlenberg', 'Nelson', 'Nicholas', 'Ohio', 'Oldham', 'Owen', 'Owsley', 'Pendleton', 'Perry', 'Pike', 'Powell', 'Pulaski', 'Robertson', 'Rockcastle', 'Rowan', 'Russell', 'Scott', 'Shelby', 'Simpson', 'Spencer', 'Taylor', 'Todd', 'Trigg', 'Trimble', 'Union', 'Warren', 'Washington', 'Wayne', 'Webster', 'Whitley', 'Wolfe', 'Woodford']


# OH
# ['Cuyahoga', 'Franklin', 'Hamilton', 'Hancock', 'Licking', 'Lorain', 'Montgomery', 'Stark']


# ['Adams', 'Allen', 'Ashland', 'Ashtabula', 'Athens', 'Auglaize', 'Belmont', 'Brown', 'Butler', 'Carroll', 'Champaign', 'Clark', 'Clermont', 'Clinton', 'Columbiana', 'Coshocton', 'Crawford', 'Cuyahoga', 'Darke', 'Defiance', 'Delaware', 'Erie', 'Fairfield', 'Fayette', 'Franklin', 'Fulton', 'Gallia', 'Geauga', 'Greene', 'Guernsey', 'Hamilton', 'Hancock', 'Hardin', 'Harrison', 'Henry', 'Highland', 'Hocking', 'Holmes', 'Huron', 'Jackson', 'Jefferson', 'Knox', 'Lake', 'Lawrence', 'Licking', 'Logan', 'Lorain', 'Lucas', 'Madison', 'Mahoning', 'Marion', 'Medina', 'Meigs', 'Mercer', 'Miami', 'Monroe', 'Montgomery', 'Morgan', 'Morrow', 'Muskingum', 'Noble', 'Ottawa', 'Paulding', 'Perry', 'Pickaway', 'Pike', 'Portage', 'Preble', 'Putnam', 'Richland', 'Ross', 'Sandusky', 'Scioto', 'Seneca', 'Shelby', 'Stark', 'Summit', 'Trumbull', 'Tuscarawas', 'Union', 'Van Wert', 'Vinton', 'Warren', 'Washington', 'Wayne', 'Williams', 'Wood', 'Wyandot']


# IN
# ['Johnson', 'Lake', 'Marion', 'Monroe', 'Wayne']


# ['Adams', 'Allen', 'Bartholomew', 'Benton', 'Blackford', 'Boone', 'Brown', 'Carroll', 'Cass', 'Clark', 'Clay', 'Clinton', 'Crawford', 'Daviess', 'DeKalb', 'Dearborn', 'Decatur', 'Delaware', 'Dubois', 'Elkhart', 'Fayette', 'Floyd', 'Fountain', 'Franklin', 'Fulton', 'Gibson', 'Grant', 'Greene', 'Hamilton', 'Hancock', 'Harrison', 'Hendricks', 'Henry', 'Howard', 'Huntington', 'Jackson', 'Jasper', 'Jay', 'Jefferson', 'Jennings', 'Johnson', 'Knox', 'Kosciusko', 'LaGrange', 'LaPorte', 'Lake', 'Lawrence', 'Madison', 'Marion', 'Marshall', 'Martin', 'Miami', 'Monroe', 'Montgomery', 'Morgan', 'Newton', 'Noble', 'Ohio', 'Orange', 'Owen', 'Parke', 'Perry', 'Pike', 'Porter', 'Posey', 'Pulaski', 'Putnam', 'Randolph', 'Ripley', 'Rush', 'Scott', 'Shelby', 'Spencer', 'St. Joseph', 'Starke', 'Steuben', 'Sullivan', 'Switzerland', 'Tippecanoe', 'Tipton', 'Union', 'Vanderburgh', 'Vermillion', 'Vigo', 'Wabash', 'Warren', 'Warrick', 'Washington', 'Wayne', 'Wells', 'White', 'Whitley']


# MI
# ['Wayne']


# ['Alcona', 'Alger', 'Allegan', 'Alpena', 'Antrim', 'Arenac', 'Baraga', 'Barry', 'Bay', 'Benzie', 'Berrien', 'Branch', 'Calhoun', 'Cass', 'Charlevoix', 'Cheboygan', 'Chippewa', 'Clare', 'Clinton', 'Crawford', 'Delta', 'Dickinson', 'Eaton', 'Emmet', 'Genesee', 'Gladwin', 'Gogebic', 'Grand Traverse', 'Gratiot', 'Hillsdale', 'Houghton', 'Huron', 'Ingham', 'Ionia', 'Iosco', 'Iron', 'Isabella', 'Jackson', 'Kalamazoo', 'Kalkaska', 'Kent', 'Keweenaw', 'Lake', 'Lapeer', 'Leelanau', 'Lenawee', 'Livingston', 'Luce', 'Mackinac', 'Macomb', 'Manistee', 'Marquette', 'Mason', 'Mecosta', 'Menominee', 'Midland', 'Missaukee', 'Monroe', 'Montcalm', 'Montmorency', 'Muskegon', 'Newaygo', 'Oakland', 'Oceana', 'Ogemaw', 'Ontonagon', 'Osceola', 'Oscoda', 'Otsego', 'Ottawa', 'Presque Isle', 'Roscommon', 'Saginaw', 'Sanilac', 'Schoolcraft', 'Shiawassee', 'St. Clair', 'St. Joseph', 'Tuscola', 'Van Buren', 'Washtenaw', 'Wayne', 'Wexford']


# IA
# ['Dubuque']


# ['Adair', 'Adams', 'Allamakee', 'Appanoose', 'Audubon', 'Benton', 'Black Hawk', 'Boone', 'Bremer', 'Buchanan', 'Buena Vista', 'Butler', 'Calhoun', 'Carroll', 'Cass', 'Cedar', 'Cerro Gordo', 'Cherokee', 'Chickasaw', 'Clarke', 'Clay', 'Clayton', 'Clinton', 'Crawford', 'Dallas', 'Davis', 'Decatur', 'Delaware', 'Des Moines', 'Dickinson', 'Dubuque', 'Emmet', 'Fayette', 'Floyd', 'Franklin', 'Fremont', 'Greene', 'Grundy', 'Guthrie', 'Hamilton', 'Hancock', 'Hardin', 'Harrison', 'Henry', 'Howard', 'Humboldt', 'Ida', 'Iowa', 'Jackson', 'Jasper', 'Jefferson', 'Johnson', 'Jones', 'Keokuk', 'Kossuth', 'Lee', 'Linn', 'Louisa', 'Lucas', 'Lyon', 'Madison', 'Mahaska', 'Marion', 'Marshall', 'Mills', 'Mitchell', 'Monona', 'Monroe', 'Montgomery', 'Muscatine', "O'Brien", 'Osceola', 'Page', 'Palo Alto', 'Plymouth', 'Pocahontas', 'Polk', 'Pottawattamie', 'Poweshiek', 'Ringgold', 'Sac', 'Scott', 'Shelby', 'Sioux', 'Story', 'Tama', 'Taylor', 'Union', 'Van Buren', 'Wapello', 'Warren', 'Washington', 'Wayne', 'Webster', 'Winnebago', 'Winneshiek', 'Woodbury', 'Worth', 'Wright']


# WI
# ['Dane', 'Kenosha', 'La Crosse', 'Manitowoc', 'Milwaukee', 'Racine', 'Winnebago']


# ['Adams', 'Ashland', 'Barron', 'Bayfield', 'Brown', 'Buffalo', 'Burnett', 'Calumet', 'Chippewa', 'Clark', 'Columbia', 'Crawford', 'Dane', 'Dodge', 'Door', 'Douglas', 'Dunn', 'Eau Claire', 'Florence', 'Fond du Lac', 'Forest', 'Grant', 'Green', 'Green Lake', 'Iowa', 'Iron', 'Jackson', 'Jefferson', 'Juneau', 'Kenosha', 'Kewaunee', 'La Crosse', 'Lafayette', 'Langlade', 'Lincoln', 'Manitowoc', 'Marathon', 'Marinette', 'Marquette', 'Menominee', 'Milwaukee', 'Monroe', 'Oconto', 'Oneida', 'Outagamie', 'Ozaukee', 'Pepin', 'Pierce', 'Polk', 'Portage', 'Price', 'Racine', 'Richland', 'Rock', 'Rusk', 'Sauk', 'Sawyer', 'Shawano', 'Sheboygan', 'St. Croix', 'Taylor', 'Trempealeau', 'Vernon', 'Vilas', 'Walworth', 'Washburn', 'Washington', 'Waukesha', 'Waupaca', 'Waushara', 'Winnebago', 'Wood']


# MN
# ['Dakota', 'Hennepin', 'Ramsey']


# ['Aitkin', 'Anoka', 'Becker', 'Beltrami', 'Benton', 'Big Stone', 'Blue Earth', 'Brown', 'Carlton', 'Carver', 'Cass', 'Chippewa', 'Chisago', 'Clay', 'Clearwater', 'Cook', 'Cottonwood', 'Crow Wing', 'Dakota', 'Dodge', 'Douglas', 'Faribault', 'Fillmore', 'Freeborn', 'Goodhue', 'Grant', 'Hennepin', 'Houston', 'Hubbard', 'Isanti', 'Itasca', 'Jackson', 'Kanabec', 'Kandiyohi', 'Kittson', 'Koochiching', 'Lac qui Parle', 'Lake', 'Lake of the Woods', 'Le Sueur', 'Lincoln', 'Lyon', 'Mahnomen', 'Marshall', 'Martin', 'McLeod', 'Meeker', 'Mille Lacs', 'Morrison', 'Mower', 'Murray', 'Nicollet', 'Nobles', 'Norman', 'Olmsted', 'Otter Tail', 'Pennington', 'Pine', 'Pipestone', 'Polk', 'Pope', 'Ramsey', 'Red Lake', 'Redwood', 'Renville', 'Rice', 'Rock', 'Roseau', 'Scott', 'Sherburne', 'Sibley', 'St. Louis', 'Stearns', 'Steele', 'Stevens', 'Swift', 'Todd', 'Traverse', 'Wabasha', 'Wadena', 'Waseca', 'Washington', 'Watonwan', 'Wilkin', 'Winona', 'Wright', 'Yellow Medicine']


# ND
# ['Cass']


# ['Adams', 'Barnes', 'Benson', 'Billings', 'Bottineau', 'Bowman', 'Burke', 'Burleigh', 'Cass', 'Cavalier', 'Dickey', 'Divide', 'Dunn', 'Eddy', 'Emmons', 'Foster', 'Golden Valley', 'Grand Forks', 'Grant', 'Griggs', 'Hettinger', 'Kidder', 'LaMoure', 'Logan', 'McHenry', 'McIntosh', 'McKenzie', 'McLean', 'Mercer', 'Morton', 'Mountrail', 'Nelson', 'Oliver', 'Pembina', 'Pierce', 'Ramsey', 'Ransom', 'Renville', 'Richland', 'Rolette', 'Sargent', 'Sheridan', 'Sioux', 'Slope', 'Stark', 'Steele', 'Stutsman', 'Towner', 'Traill', 'Walsh', 'Ward', 'Wells', 'Williams']


# MT
# ['Flathead', 'Gallatin', 'Lewis and Clark', 'Missoula']


# ['Beaverhead', 'Big Horn', 'Blaine', 'Broadwater', 'Carbon', 'Carter', 'Cascade', 'Chouteau', 'Custer', 'Daniels', 'Dawson', 'Deer Lodge', 'Fallon', 'Fergus', 'Flathead', 'Gallatin', 'Garfield', 'Glacier', 'Golden Valley', 'Granite', 'Hill', 'Jefferson', 'Judith Basin', 'Lake', 'Lewis and Clark', 'Liberty', 'Lincoln', 'Madison', 'McCone', 'Meagher', 'Mineral', 'Missoula', 'Musselshell', 'Park', 'Petroleum', 'Phillips', 'Pondera', 'Powder River', 'Powell', 'Prairie', 'Ravalli', 'Richland', 'Roosevelt', 'Rosebud', 'Sanders', 'Sheridan', 'Silver Bow', 'Stillwater', 'Sweet Grass', 'Teton', 'Toole', 'Treasure', 'Valley', 'Wheatland', 'Wibaux', 'Yellowstone']


# IL
# ['Cook', 'DuPage', 'Lake', 'Winnebago']


# ['Adams', 'Alexander', 'Bond', 'Boone', 'Brown', 'Bureau', 'Calhoun', 'Carroll', 'Cass', 'Champaign', 'Christian', 'Clark', 'Clay', 'Clinton', 'Coles', 'Cook', 'Crawford', 'Cumberland', 'De Witt', 'DeKalb', 'Douglas', 'DuPage', 'Edgar', 'Edwards', 'Effingham', 'Fayette', 'Ford', 'Franklin', 'Fulton', 'Gallatin', 'Greene', 'Grundy', 'Hamilton', 'Hancock', 'Hardin', 'Henderson', 'Henry', 'Iroquois', 'Jackson', 'Jasper', 'Jefferson', 'Jersey', 'Jo Daviess', 'Johnson', 'Kane', 'Kankakee', 'Kendall', 'Knox', 'LaSalle', 'Lake', 'Lawrence', 'Lee', 'Livingston', 'Logan', 'Macon', 'Macoupin', 'Madison', 'Marion', 'Marshall', 'Mason', 'Massac', 'McDonough', 'McHenry', 'McLean', 'Menard', 'Mercer', 'Monroe', 'Montgomery', 'Morgan', 'Moultrie', 'Ogle', 'Peoria', 'Perry', 'Piatt', 'Pike', 'Pope', 'Pulaski', 'Putnam', 'Randolph', 'Richland', 'Rock Island', 'Saline', 'Sangamon', 'Schuyler', 'Scott', 'Shelby', 'St. Clair', 'Stark', 'Stephenson', 'Tazewell', 'Union', 'Vermilion', 'Wabash', 'Warren', 'Washington', 'Wayne', 'White', 'Whiteside', 'Will', 'Williamson', 'Winnebago', 'Woodford']


# MO
# ['Boone', 'Jackson', 'Jasper', 'Saint Louis', 'Saint Louis City']


# ['Adair', 'Andrew', 'Atchison', 'Audrain', 'Barry', 'Barton', 'Bates', 'Benton', 'Bollinger', 'Boone', 'Buchanan', 'Butler', 'Caldwell', 'Callaway', 'Camden', 'Cape Girardeau', 'Carroll', 'Carter', 'Cass', 'Cedar', 'Chariton', 'Christian', 'Clark', 'Clay', 'Clinton', 'Cole', 'Cooper', 'Crawford', 'Dade', 'Dallas', 'Daviess', 'DeKalb', 'Dent', 'Douglas', 'Dunklin', 'Franklin', 'Gasconade', 'Gentry', 'Greene', 'Grundy', 'Harrison', 'Henry', 'Hickory', 'Holt', 'Howard', 'Howell', 'Iron', 'Jackson', 'Jasper', 'Jefferson', 'Johnson', 'Knox', 'Laclede', 'Lafayette', 'Lawrence', 'Lewis', 'Lincoln', 'Linn', 'Livingston', 'Macon', 'Madison', 'Maries', 'Marion', 'McDonald', 'Mercer', 'Miller', 'Mississippi', 'Moniteau', 'Monroe', 'Montgomery', 'Morgan', 'New Madrid', 'Newton', 'Nodaway', 'Oregon', 'Osage', 'Ozark', 'Pemiscot', 'Perry', 'Pettis', 'Phelps', 'Pike', 'Platte', 'Polk', 'Pulaski', 'Putnam', 'Ralls', 'Randolph', 'Ray', 'Reynolds', 'Ripley', 'Saline', 'Schuyler', 'Scotland', 'Scott', 'Shannon', 'Shelby', 'St. Charles', 'St. Clair', 'St. Francois', 'St. Louis', 'St. Louis city', 'Ste. Genevieve', 'Stoddard', 'Stone', 'Sullivan', 'Taney', 'Texas', 'Vernon', 'Warren', 'Washington', 'Wayne', 'Webster', 'Worth', 'Wright']


# KS
# ['Johnson', 'Riley', 'Sedgwick', 'Shawnee']


# ['Allen', 'Anderson', 'Atchison', 'Barber', 'Barton', 'Bourbon', 'Brown', 'Butler', 'Chase', 'Chautauqua', 'Cherokee', 'Cheyenne', 'Clark', 'Clay', 'Cloud', 'Coffey', 'Comanche', 'Cowley', 'Crawford', 'Decatur', 'Dickinson', 'Doniphan', 'Douglas', 'Edwards', 'Elk', 'Ellis', 'Ellsworth', 'Finney', 'Ford', 'Franklin', 'Geary', 'Gove', 'Graham', 'Grant', 'Gray', 'Greeley', 'Greenwood', 'Hamilton', 'Harper', 'Harvey', 'Haskell', 'Hodgeman', 'Jackson', 'Jefferson', 'Jewell', 'Johnson', 'Kearny', 'Kingman', 'Kiowa', 'Labette', 'Lane', 'Leavenworth', 'Lincoln', 'Linn', 'Logan', 'Lyon', 'Marion', 'Marshall', 'McPherson', 'Meade', 'Miami', 'Mitchell', 'Montgomery', 'Morris', 'Morton', 'Nemaha', 'Neosho', 'Ness', 'Norton', 'Osage', 'Osborne', 'Ottawa', 'Pawnee', 'Phillips', 'Pottawatomie', 'Pratt', 'Rawlins', 'Reno', 'Republic', 'Rice', 'Riley', 'Rooks', 'Rush', 'Russell', 'Saline', 'Scott', 'Sedgwick', 'Seward', 'Shawnee', 'Sheridan', 'Sherman', 'Smith', 'Stafford', 'Stanton', 'Stevens', 'Sumner', 'Thomas', 'Trego', 'Wabaunsee', 'Wallace', 'Washington', 'Wichita', 'Wilson', 'Woodson', 'Wyandotte']


# AR
# ['Benton', 'Pulaski', 'Washington']


# ['Arkansas', 'Ashley', 'Baxter', 'Benton', 'Boone', 'Bradley', 'Calhoun', 'Carroll', 'Chicot', 'Clark', 'Clay', 'Cleburne', 'Cleveland', 'Columbia', 'Conway', 'Craighead', 'Crawford', 'Crittenden', 'Cross', 'Dallas', 'Desha', 'Drew', 'Faulkner', 'Franklin', 'Fulton', 'Garland', 'Grant', 'Greene', 'Hempstead', 'Hot Spring', 'Howard', 'Independence', 'Izard', 'Jackson', 'Jefferson', 'Johnson', 'Lafayette', 'Lawrence', 'Lee', 'Lincoln', 'Little River', 'Logan', 'Lonoke', 'Madison', 'Marion', 'Miller', 'Mississippi', 'Monroe', 'Montgomery', 'Nevada', 'Newton', 'Ouachita', 'Perry', 'Phillips', 'Pike', 'Poinsett', 'Polk', 'Pope', 'Prairie', 'Pulaski', 'Randolph', 'Saline', 'Scott', 'Searcy', 'Sebastian', 'Sevier', 'Sharp', 'St. Francis', 'Stone', 'Union', 'Van Buren', 'Washington', 'White', 'Woodruff', 'Yell']


# OK
# ['Cleveland', 'Oklahoma']


# ['Adair', 'Alfalfa', 'Atoka', 'Beaver', 'Beckham', 'Blaine', 'Bryan', 'Caddo', 'Canadian', 'Carter', 'Cherokee', 'Choctaw', 'Cimarron', 'Cleveland', 'Coal', 'Comanche', 'Cotton', 'Craig', 'Creek', 'Custer', 'Delaware', 'Dewey', 'Ellis', 'Garfield', 'Garvin', 'Grady', 'Grant', 'Greer', 'Harmon', 'Harper', 'Haskell', 'Hughes', 'Jackson', 'Jefferson', 'Johnston', 'Kay', 'Kingfisher', 'Kiowa', 'Latimer', 'Le Flore', 'Lincoln', 'Logan', 'Love', 'Major', 'Marshall', 'Mayes', 'McClain', 'McCurtain', 'McIntosh', 'Murray', 'Muskogee', 'Noble', 'Nowata', 'Okfuskee', 'Oklahoma', 'Okmulgee', 'Osage', 'Ottawa', 'Pawnee', 'Payne', 'Pittsburg', 'Pontotoc', 'Pottawatomie', 'Pushmataha', 'Roger Mills', 'Rogers', 'Seminole', 'Sequoyah', 'Stephens', 'Texas', 'Tillman', 'Tulsa', 'Wagoner', 'Washington', 'Washita', 'Woods', 'Woodward']


# TX
# ['Bell', 'Bexar', 'Brazoria', 'Brazos', 'Cameron', 'Collin', 'Comal', 'Coryell', 'Dallas', 'Denton', 'El Paso', 'Galveston', 'Harris', 'Hays', 'Montgomery', 'Smith', 'Tarrant', 'Travis', 'Williamson']


# ['Anderson', 'Andrews', 'Angelina', 'Aransas', 'Archer', 'Armstrong', 'Atascosa', 'Austin', 'Bailey', 'Bandera', 'Bastrop', 'Baylor', 'Bee', 'Bell', 'Bexar', 'Blanco', 'Borden', 'Bosque', 'Bowie', 'Brazoria', 'Brazos', 'Brewster', 'Briscoe', 'Brooks', 'Brown', 'Burleson', 'Burnet', 'Caldwell', 'Calhoun', 'Callahan', 'Cameron', 'Camp', 'Carson', 'Cass', 'Castro', 'Chambers', 'Cherokee', 'Childress', 'Clay', 'Cochran', 'Coke', 'Coleman', 'Collin', 'Collingsworth', 'Colorado', 'Comal', 'Comanche', 'Concho', 'Cooke', 'Coryell', 'Cottle', 'Crane', 'Crockett', 'Crosby', 'Culberson', 'Dallam', 'Dallas', 'Dawson', 'DeWitt', 'Deaf Smith', 'Delta', 'Denton', 'Dickens', 'Dimmit', 'Donley', 'Duval', 'Eastland', 'Ector', 'Edwards', 'El Paso', 'Ellis', 'Erath', 'Falls', 'Fannin', 'Fayette', 'Fisher', 'Floyd', 'Foard', 'Fort Bend', 'Franklin', 'Freestone', 'Frio', 'Gaines', 'Galveston', 'Garza', 'Gillespie', 'Glasscock', 'Goliad', 'Gonzales', 'Gray', 'Grayson', 'Gregg', 'Grimes', 'Guadalupe', 'Hale', 'Hall', 'Hamilton', 'Hansford', 'Hardeman', 'Hardin', 'Harris', 'Harrison', 'Hartley', 'Haskell', 'Hays', 'Hemphill', 'Henderson', 'Hidalgo', 'Hill', 'Hockley', 'Hood', 'Hopkins', 'Houston', 'Howard', 'Hudspeth', 'Hunt', 'Hutchinson', 'Irion', 'Jack', 'Jackson', 'Jasper', 'Jeff Davis', 'Jefferson', 'Jim Hogg', 'Jim Wells', 'Johnson', 'Jones', 'Karnes', 'Kaufman', 'Kendall', 'Kenedy', 'Kent', 'Kerr', 'Kimble', 'King', 'Kinney', 'Kleberg', 'Knox', 'La Salle', 'Lamar', 'Lamb', 'Lampasas', 'Lavaca', 'Lee', 'Leon', 'Liberty', 'Limestone', 'Lipscomb', 'Live Oak', 'Llano', 'Loving', 'Lubbock', 'Lynn', 'Madison', 'Marion', 'Martin', 'Mason', 'Matagorda', 'Maverick', 'McCulloch', 'McLennan', 'McMullen', 'Medina', 'Menard', 'Midland', 'Milam', 'Mills', 'Mitchell', 'Montague', 'Montgomery', 'Moore', 'Morris', 'Motley', 'Nacogdoches', 'Navarro', 'Newton', 'Nolan', 'Nueces', 'Ochiltree', 'Oldham', 'Orange', 'Palo Pinto', 'Panola', 'Parker', 'Parmer', 'Pecos', 'Polk', 'Potter', 'Presidio', 'Rains', 'Randall', 'Reagan', 'Real', 'Red River', 'Reeves', 'Refugio', 'Roberts', 'Robertson', 'Rockwall', 'Runnels', 'Rusk', 'Sabine', 'San Augustine', 'San Jacinto', 'San Patricio', 'San Saba', 'Schleicher', 'Scurry', 'Shackelford', 'Shelby', 'Sherman', 'Smith', 'Somervell', 'Starr', 'Stephens', 'Sterling', 'Stonewall', 'Sutton', 'Swisher', 'Tarrant', 'Taylor', 'Terrell', 'Terry', 'Throckmorton', 'Titus', 'Tom Green', 'Travis', 'Trinity', 'Tyler', 'Upshur', 'Upton', 'Uvalde', 'Val Verde', 'Van Zandt', 'Victoria', 'Walker', 'Waller', 'Ward', 'Washington', 'Webb', 'Wharton', 'Wheeler', 'Wichita', 'Wilbarger', 'Willacy', 'Williamson', 'Wilson', 'Winkler', 'Wise', 'Wood', 'Yoakum', 'Young', 'Zapata', 'Zavala']


# CO
# ['Adams', 'Arapahoe', 'Boulder', 'Denver', 'Douglas', 'El Paso', 'Jefferson']


# ['Adams', 'Alamosa', 'Arapahoe', 'Archuleta', 'Baca', 'Bent', 'Boulder', 'Broomfield', 'Chaffee', 'Cheyenne', 'Clear Creek', 'Conejos', 'Costilla', 'Crowley', 'Custer', 'Delta', 'Denver', 'Dolores', 'Douglas', 'Eagle', 'El Paso', 'Elbert', 'Fremont', 'Garfield', 'Gilpin', 'Grand', 'Gunnison', 'Hinsdale', 'Huerfano', 'Jackson', 'Jefferson', 'Kiowa', 'Kit Carson', 'La Plata', 'Lake', 'Larimer', 'Las Animas', 'Lincoln', 'Logan', 'Mesa', 'Mineral', 'Moffat', 'Montezuma', 'Montrose', 'Morgan', 'Otero', 'Ouray', 'Park', 'Phillips', 'Pitkin', 'Prowers', 'Pueblo', 'Rio Blanco', 'Rio Grande', 'Routt', 'Saguache', 'San Juan', 'San Miguel', 'Sedgwick', 'Summit', 'Teller', 'Washington', 'Weld', 'Yuma']


# UT
# ['Salt Lake', 'Weber']


# ['Beaver', 'Box Elder', 'Cache', 'Carbon', 'Daggett', 'Davis', 'Duchesne', 'Emery', 'Garfield', 'Grand', 'Iron', 'Juab', 'Kane', 'Millard', 'Morgan', 'Piute', 'Rich', 'Salt Lake', 'San Juan', 'Sanpete', 'Sevier', 'Summit', 'Tooele', 'Uintah', 'Utah', 'Wasatch', 'Washington', 'Wayne', 'Weber']


# AZ
# ['Maricopa', 'Pima', 'Yavapai']


# ['Apache', 'Cochise', 'Coconino', 'Gila', 'Graham', 'Greenlee', 'La Paz', 'Maricopa', 'Mohave', 'Navajo', 'Pima', 'Pinal', 'Santa Cruz', 'Yavapai', 'Yuma']


# NM
# ['Bernalillo']


# ['Bernalillo', 'Catron', 'Chaves', 'Cibola', 'Colfax', 'Curry', 'De Baca', 'Doña Ana', 'Eddy', 'Grant', 'Guadalupe', 'Harding', 'Hidalgo', 'Lea', 'Lincoln', 'Los Alamos', 'Luna', 'McKinley', 'Mora', 'Otero', 'Quay', 'Rio Arriba', 'Roosevelt', 'San Juan', 'San Miguel', 'Sandoval', 'Santa Fe', 'Sierra', 'Socorro', 'Taos', 'Torrance', 'Union', 'Valencia']


# NV
# ['Clark', 'Washoe']


# ['Carson City', 'Churchill', 'Clark', 'Douglas', 'Elko', 'Esmeralda', 'Eureka', 'Humboldt', 'Lander', 'Lincoln', 'Lyon', 'Mineral', 'Nye', 'Pershing', 'Storey', 'Washoe', 'White Pine']


# CA
# ['Alameda', 'Butte', 'Contra Costa', 'El Dorado', 'Kern', 'Los Angeles', 'Marin', 'Monterey', 'Orange', 'Placer', 'Riverside', 'Sacramento', 'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin', 'San Mateo', 'Santa Clara', 'Ventura']


# ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa', 'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino', 'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito', 'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin', 'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo', 'Yuba']


# HI
# ['Honolulu']


# ['Hawaii', 'Honolulu', 'Kauai', 'Maui']


# OR
# ['Clackamas', 'Jackson', 'Lane', 'Multnomah', 'Washington']


# ['Baker', 'Benton', 'Clackamas', 'Clatsop', 'Columbia', 'Coos', 'Crook', 'Curry', 'Deschutes', 'Douglas', 'Gilliam', 'Grant', 'Harney', 'Hood River', 'Jackson', 'Jefferson', 'Josephine', 'Klamath', 'Lake', 'Lane', 'Lincoln', 'Linn', 'Malheur', 'Marion', 'Morrow', 'Multnomah', 'Polk', 'Sherman', 'Tillamook', 'Umatilla', 'Union', 'Wallowa', 'Wasco', 'Washington', 'Wheeler', 'Yamhill']


# WA
# ['King', 'Pierce', 'Snohomish', 'Spokane']


# ['Adams', 'Asotin', 'Benton', 'Chelan', 'Clallam', 'Clark', 'Columbia', 'Cowlitz', 'Douglas', 'Ferry', 'Franklin', 'Garfield', 'Grant', 'Grays Harbor', 'Island', 'Jefferson', 'King', 'Kitsap', 'Kittitas', 'Klickitat', 'Lewis', 'Lincoln', 'Mason', 'Okanogan', 'Pacific', 'Pend Oreille', 'Pierce', 'San Juan', 'Skagit', 'Skamania', 'Snohomish', 'Spokane', 'Stevens', 'Thurston', 'Wahkiakum', 'Walla Walla', 'Whatcom', 'Whitman', 'Yakima']


# AK
# ['Anchorage', 'Matanuska Susitna']


# ['Aleutian Islands', 'Aleutians East', 'Aleutians West', 'Anchorage', 'Bethel', 'Bristol Bay', 'Denali', 'Dillingham', 'Fairbanks North Star', 'Haines', 'Hoonah-Angoon', 'Juneau City and', 'Kenai Peninsula', 'Ketchikan Gateway', 'Kobuk', 'Kodiak Island', 'Lake and Peninsula', 'Matanuska-Susitna', 'Nome', 'North Slope', 'Northwest Arctic', 'Petersburg', 'Prince of Wales-Hyder', 'Prince of Wales-Outer Ketchikan', 'Sitka', 'Skagway', 'Skagway-Hoonah-Angoon', 'Skagway-Yakutat-Angoon', 'Southeast Fairbanks', 'Valdez-Cordova', 'Wrangell', 'Wrangell-Petersburg', 'Yakutat', 'Yukon-Koyukuk']

